## Get the image link and names from google

In [1]:
import os
import re
import time
import json
import pickle
import threading
from datetime import datetime
from queue import Queue
from io import BytesIO
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import numpy as np
import requests
from PIL import Image, ImageFile
import pytesseract
import gdown
# Google Drive and Auth
from pydrive2.auth import GoogleAuth as GoogleAuth2
from pydrive2.drive import GoogleDrive as GoogleDrive2
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.service_account import ServiceAccountCredentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import google.auth
from google.auth.transport.requests import AuthorizedSession
from googleapiclient.discovery import build
from googleapiclient.http import build_http
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [2]:
# Define scopes and folder ID
SCOPES = ["https://www.googleapis.com/auth/drive"]
FOLDER_ID_2016_2017 = "0AB47zEhxmVHPUk9PVA"  
FOLDER_ID_2017_2018 = "0ALVHtXWwU7NPUk9PVA"
FOLDER_ID_2018_2019_1 = "0AAV-GgAjGbW2Uk9PVA"
FOLDER_ID_2018_2019_2 = "0AKW1tx0XOaaJUk9PVA"

# # Authenticate
# gauth = GoogleAuth()
# gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name('../../credentials.json', SCOPES)
# drive = GoogleDrive(gauth)

In [3]:
# # === Global Variables ===
# SCOPES = ['https://www.googleapis.com/auth/drive.readonly']
# folder_queue = Queue()
# final_links = []  # Now a list of records instead of nested dict

# # === Step 1: Authentication ===
# def authenticate_drive_api():
#     creds = None
#     if os.path.exists('token.pickle'):
#         with open('token.pickle', 'rb') as token:
#             creds = pickle.load(token)
#     if not creds or not creds.valid:
#         if creds and creds.expired and creds.refresh_token:
#             creds.refresh(Request())
#         else:
#             flow = InstalledAppFlow.from_client_secrets_file('client_secrets.json', SCOPES)
#             creds = flow.run_local_server(port=0)
#         with open('token.pickle', 'wb') as token:
#             pickle.dump(creds, token)
#     service = build('drive', 'v3', credentials=creds)
#     return service

# # === Step 2: Worker Function ===
# def worker():
#     service = authenticate_drive_api()
#     while True:
#         context = folder_queue.get()
#         if context is None:
#             break
#         crawl_folder(service, context)
#         folder_queue.task_done()

# # === Step 3: Folder Crawler ===
# def crawl_folder(service, context):
#     folder_id, current_path = context  # (folder_id, full_path)

#     query = f"'{folder_id}' in parents and trashed=false"
#     page_token = None
#     while True:
#         response = service.files().list(
#             q=query,
#             spaces='drive',
#             fields='nextPageToken, files(id, name, mimeType)',
#             supportsAllDrives=True,
#             includeItemsFromAllDrives=True,
#             pageToken=page_token
#         ).execute()

#         for file in response.get('files', []):
#             mime_type = file['mimeType']
#             file_id = file['id']
#             title = file['name']

#             if mime_type == 'application/vnd.google-apps.folder':
#                 # It's a folder: recurse into it
#                 next_path = os.path.join(current_path, title)
#                 folder_queue.put((file_id, next_path))

#             elif mime_type.startswith('image/'):
#                 view_link = f"https://drive.google.com/file/d/{file_id}/view"
#                 final_links.append({
#                     "filename": title,
#                     "filepath": view_link,
#                     "directory": current_path.replace("\\", "/")  # Make sure slashes are correct
#                 })

#         page_token = response.get('nextPageToken', None)
#         if page_token is None:
#             break

# # === Step 4: Collect Image Links in Parallel ===
# def collect_image_links_parallel(start_folder_id, num_threads=10):
#     threads = []
#     for _ in range(num_threads):
#         t = threading.Thread(target=worker)
#         t.start()
#         threads.append(t)

#     # Start with root folder (path is empty string)
#     folder_queue.put((start_folder_id, ""))

#     folder_queue.join()

#     # Stop workers
#     for _ in threads:
#         folder_queue.put(None)
#     for t in threads:
#         t.join()

#     return final_links

# # === Step 5: Main Execution ===
# if __name__ == "__main__":
#     service = authenticate_drive_api()

#     # Your Folder ID
#     FOLDER_ID_2018_2019_2 = "0AKW1tx0XOaaJUk9PVA"

#     all_image_links = collect_image_links_parallel(FOLDER_ID_2018_2019_2)

#     print(f"\n✅ Found {len(all_image_links)} images!")

#     # Save as JSON
#     with open("all_image_links_flat.json", "w") as f:
#         json.dump(all_image_links, f, indent=4)

#     print("\n✅ Saved to all_image_links_flat.json!")

## Clean Master File

In [ ]:
master_file = pd.read_csv("wildcam_fulldata_2019.csv", parse_dates=['date', 'datetime'])

master_file = master_file.rename(columns=lambda x: x.strip())

print(f"✅ Rows before cleaning 'filename': {len(master_file)}")

master_file['filename'] = master_file['filename'].str.strip()
master_file['species'] = master_file['species'].str.capitalize()

# Step 1: Drop rows where filename is actually NaN (not a string)
master_file = master_file[master_file['filename'].notna()]

# Step 2: Now strip and drop empty or whitespace-only filenames
master_file['filename'] = master_file['filename'].str.strip()
master_file = master_file[master_file['filename'] != '']

# Confirm
print(f"✅ Remaining rows after cleaning 'filename': {len(master_file)}")

# Drop exact duplicates across all columns, keep only the first occurrence
master_file = master_file.drop_duplicates(keep='first')

# Confirm the result
print(f"✅ Rows remaining after dropping exact duplicates: {len(master_file)}")

In [ ]:
# master_file_first_half = master_file[~master_file['year_start'].isin([2018, 2019])]
# master_file_second_half = master_file[master_file['year_start'].isin([2018, 2019])]
# print(f"✅ Total Number of Rows in master_file_first_half: {len(master_file_first_half)}")
# print(f"✅ Total Number of Rows in master_file_second_half: {len(master_file_second_half)}")

# total_split = len(master_file_first_half) + len(master_file_second_half)
# original_total = len(master_file)

# if total_split == original_total:
#     print(f"✅ Split is complete: {total_split} rows match the original {original_total}.")
# else:
#     print(f"❌ Mismatch: split total = {total_split}, original = {original_total}.")

## Clean Additional File

In [ ]:
add_file = pd.read_csv("wildcam_year3_additionaldata.csv", parse_dates=['DateTimeOriginal', 'DateTimeCorrected'])

add_file = add_file.rename(columns=lambda x: x.lower())
add_file

## Combine JSON files into Pandas DataFrame

In [ ]:
def json_to_df(json_path):
    """
    Converts a nested JSON {topic: {id: text}} into a flat pandas DataFrame.

    Args:
        json_path (str): Path to the JSON file.

    Returns:
        pd.DataFrame: DataFrame with columns ['topic', 'id', 'text']
    """
    with open(json_path, 'r') as f:
        data = json.load(f)

    return pd.DataFrame(data)

In [ ]:
df_2016_2017 = json_to_df("all_image_links_2016_2017.json")
df_2017_2018 = json_to_df("all_image_links_2017_2018.json")
df_2018_2019_1 = json_to_df("all_image_links_2018_2019_1.json")
df_2018_2019_2 = json_to_df("all_image_links_2018_2019_2.json")

In [ ]:
full_df = pd.concat([df_2016_2017, df_2017_2018, df_2018_2019_1, df_2018_2019_2])

print(f"✅ Total Number of Rows: {len(full_df)}")

In [ ]:
# First, make sure the column exists and is clean
full_df['filename'] = full_df['filename'].astype(str).str.strip()

# Step 1: Count how many match the _original pattern
num_originals = full_df[full_df['filename'].str.endswith('_original')].shape[0]
print(f"🧮 Rows ending with _original.JPG: {num_originals}")

# Step 2: Remove those rows
full_df = full_df[~full_df['filename'].str.endswith('_original')]

# Step 3: Check the new shape
print(f"✅ Remaining rows after 'original' removal: {len(full_df)}")

In [ ]:
# Clean 'directory'
full_df.loc[:, 'directory'] = full_df['directory'].astype(str).str.strip()

# Create 'site' and 'species'
full_df.loc[:, 'site'] = full_df['directory'].apply(lambda x: x.split('/')[0])
full_df.loc[:, 'species'] = full_df['directory'].apply(lambda x: x.split('/')[-1])

In [ ]:
def standardize_species(name):
    if pd.isna(name):
        return name

    name = name.strip()

    # 1. Remove trailing numeric suffixes like " 2"
    name = re.sub(r"\s+2$", "", name)

    # 2. Replace inconsistent naming
    if name == "Mongoose_white tailed":
        name = "Mongoose_white_tailed"
        
    if name == "Mongoose_large_gray":
        name = "Mongoose_large_grey"
        
    if name == "Samango":
        name = "Samango_monkey"
        
    if name == "Mongoose_other":
        name = "Mongoose"

    if name == "Vervet":
        name = "Vervet_monkey"
        
    if name == "Hippo":
        name = "Hippopotamus"
        
    if name == "Hornbill_ground":
        name = "Ground_hornbill"
        
    if name in ['Snake', 'Lizard', 'Monitor_lizard', 'Reptile']:
        name = 'Reptile_amphibian'

    # 3. Remove '_unknown' from names like 'Mongoose_unknown', 'Duiker_unknown'
    name = re.sub(r"_unknown$", "", name)

    return name

In [ ]:
# full_df = extract_datetime_from_filename(full_df)
# Apply the updated function
full_df['species'] = full_df['species'].apply(standardize_species)

full_df.to_csv('full_df.csv', index=False)  

In [ ]:
missing_dt = full_df[full_df[['date', 'time']].isnull().any(axis=1)]
missing_dt

In [ ]:
import os
os.environ["GOOGLE_API_USE_MTLS_ENDPOINT"] = "never"
os.environ["GOOGLE_API_USE_UNCACHED_DISCOVERY_DOCS"] = "true"

In [ ]:
# Find duplicated filepath values
duplicated_filepaths = full_df[full_df.duplicated(subset='filepath', keep=False)]

# Count them
print(f"🔍 Number of rows with duplicated filepath: {len(duplicated_filepaths)}")

# Preview some
duplicated_filepaths.sort_values('filepath').head()

In [ ]:
def simplify_directory(path):
    if pd.isna(path):
        return path
    parts = path.strip("/").split("/")
    return "/".join(parts[-2:]) if len(parts) >= 2 else path

# Apply the function to the directory column
master_file['directory'] = master_file['directory'].apply(simplify_directory)

In [ ]:
# Step 1: List columns from master_file to include (exclude 'directory' but keep join keys)
# cols_to_merge = [col for col in master_file_first_half.columns if col != 'directory' or col in ['filename', 'site', 'species', 'year_start']]

# Step 2: Perform the merge
merged_df_1 = pd.merge(
    full_df,
    master_file,
    on=['filename', 'directory'],
    how='inner'
)
merged_df_1

## Match jsons with master file

In [ ]:
# # step 1

# python -m megadetector.detection.run_detector_batch 
# MDV5A 
# ~/Desktop/Kaitlyn_Catalyst/ct_classifier/datasets/CaltechCT/eccv_18_all_images_sm 
# ~/Desktop/Kaitlyn_Catalyst/ct_classifier/datasets/CaltechCT/eccv_18_all_images_sm.json 
# --output_relative_filenames 
# --recursive 
# --checkpoint_frequency 10000 
# --quiet


In [ ]:
# # step 2

# python -m megadetector.separate_detections_into_folders.py 
# Desktop/Kaitlyn_Catalyst/ct_classifier/datasets/CaltechCT/eccv_18_all_images_sm.json 
# Desktop/Kaitlyn_Catalyst/ct_classifier/datasets/CaltechCT/eccv_18_all_images_sm 
# Desktop/Kaitlyn_Catalyst/ct_classifier/datasets/CaltechCT/separated_images